In [64]:
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
import requests
from src.libs.utils.functions import *

In [65]:
countries_req=requests.get('https://api.mercadolibre.com/sites')
countries=countries_req.json()
country_selected=filter_list_of_dicts_by_string_in_key(list_of_dicts=countries,key_name='name',string_value='Col')
print(country_selected)

[{'default_currency_id': 'COP', 'id': 'MCO', 'name': 'Colombia'}]


In [66]:
country_id=country_selected[0].get('id')
cats = requests.get(f'https://api.mercadolibre.com/sites/{country_id}/categories')
categorias=cats.json()
cat_selected=filter_list_of_dicts_by_string_in_key(list_of_dicts=categorias,key_name='name',string_value='Celular')
print(cat_selected)

[{'id': 'MCO1051', 'name': 'Celulares y Teléfonos'}]


In [67]:
cat_id = cat_selected[0].get('id')
offset = range(0, 951, 50)
df_items=pd.DataFrame()
for i in offset:
    url = f'https://api.mercadolibre.com/sites/MCO/search?category={cat_id}&offset={i}'
    itms = requests.get(url)
    items_tmp=itms.json()
    df_items=pd.concat([df_items,pd.DataFrame(items_tmp.get('results'))],axis=0,ignore_index=True)
df_items.rename(columns={'id':'item_id'},inplace=True)    

In [68]:
df_items.columns

Index(['item_id', 'title', 'condition', 'thumbnail_id', 'catalog_product_id',
       'listing_type_id', 'permalink', 'buying_mode', 'site_id', 'category_id',
       'domain_id', 'thumbnail', 'currency_id', 'order_backend', 'price',
       'original_price', 'sale_price', 'sold_quantity', 'available_quantity',
       'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago', 'tags',
       'shipping', 'stop_time', 'seller', 'seller_address', 'address',
       'attributes', 'installments', 'winner_item_id', 'catalog_listing',
       'discounts', 'promotions', 'inventory_id', 'official_store_name',
       'differential_pricing', 'variation_filters', 'variations_data'],
      dtype='object')

In [69]:
df_items=expand_dict_column(df_items,'seller')
df_items=expand_dict_column(df_items,'installments')
df_items.rename(columns={'id':'seller_id'},inplace=True) 

In [76]:
print(df_items.shape)
print(df_items.columns)
print(df_items.dtypes)

(1000, 85)
Index(['item_id', 'title', 'condition', 'thumbnail_id', 'catalog_product_id',
       'listing_type_id', 'permalink', 'buying_mode', 'site_id', 'category_id',
       'domain_id', 'thumbnail', 'currency_id', 'order_backend', 'price',
       'original_price', 'sale_price', 'sold_quantity', 'available_quantity',
       'official_store_id', 'use_thumbnail_id', 'accepts_mercadopago', 'tags',
       'shipping', 'stop_time', 'seller_address', 'address', 'attributes',
       'winner_item_id', 'catalog_listing', 'discounts', 'promotions',
       'inventory_id', 'official_store_name', 'differential_pricing',
       'variation_filters', 'variations_data', 'seller_id', 'nickname',
       'car_dealer', 'real_estate_agency', '_', 'registration_date', 'tags',
       'car_dealer_logo', 'permalink', 'seller_reputation.level_id',
       'seller_reputation.power_seller_status',
       'seller_reputation.transactions.canceled',
       'seller_reputation.transactions.completed',
       'seller_re

In [151]:
# df_items['differential_pricing'].loc[~df_items.differential_pricing.isnull()]
#df_items.quantity.value_counts(dropna=False)
#df_items.groupby(['seller_id','quantity']).agg(cuotas=('quantity','mean')).reset_index(drop=False).sort_values(by='cuotas').groupby(['seller_id']).count().sort_values(by='cuotas')
# df_items.promotions.value_counts(dropna=False)
# df_items.groupby(['seller_id'])['seller_reputation.transactions.period'].agg('nunique').sort_values(ascending=False)
df_items['seller_reputation.transactions.period'].value_counts(dropna=False)

seller_reputation.transactions.period
historic    1000
Name: count, dtype: int64

In [153]:
df_items[['item_id','seller_id','registration_date','seller_reputation.metrics.sales.period',
       'seller_reputation.metrics.sales.completed']].head(3)

,item_id,seller_id,registration_date,seller_reputation.metrics.sales.period,seller_reputation.metrics.sales.completed
0,MCO877397746,162681369,2014-07-16T15:56:16.000-04:00,60 days,4606
1,MCO1349872512,162681369,2014-07-16T15:56:16.000-04:00,60 days,4606
2,MCO888584026,162681369,2014-07-16T15:56:16.000-04:00,60 days,4606


In [152]:
(
    df_items
    .assign(
        discount=lambda df: np.where(( (df['price']<df['original_price']) & (df.original_price.notna()) ),100*(1-df['price']/df['original_price']), 0))
    .groupby(["seller_id","seller_reputation.power_seller_status",'seller_reputation.transactions.period'],dropna=False)
    .agg(
        n_items=('item_id','count'),
        n_categories=("category_id",'nunique'),
        sold_items=('sold_quantity','sum'),
        mean_intallments=('quantity','mean'),
        q10_discount=('discount',lambda x: x.quantile(0.1)),
        q50_discount=('discount',lambda x: x.quantile(0.5)),
        q90_discount=('discount',lambda x: x.quantile(0.9)),
        total_transactions=('seller_reputation.transactions.total','mean'),
        completed_transactions=('seller_reputation.transactions.completed','mean'),
        canceled_transactions=('seller_reputation.transactions.canceled','mean'),
        negative_rating=('seller_reputation.transactions.ratings.negative','mean'),
        neutral_rating=('seller_reputation.transactions.ratings.neutral','mean'),
        positive_rating=('seller_reputation.transactions.ratings.positive','mean')
        )
    .reset_index(drop=False)       
    )   

,seller_id,seller_reputation.power_seller_status,seller_reputation.transactions.period,n_items,n_categories,sold_items,mean_intallments,q10_discount,q50_discount,q90_discount,total_transactions,completed_transactions,canceled_transactions,negative_rating,neutral_rating,positive_rating
0,1066358,NaN,historic,1,1,2,12.000000,0.000000,0.000000,0.000000,1183.0,1111.0,72.0,0.01,0.02,0.97
1,3637758,gold,historic,1,1,5,36.000000,0.000000,0.000000,0.000000,3087.0,2934.0,153.0,0.04,0.00,0.96
2,4944181,NaN,historic,1,1,0,36.000000,10.000000,10.000000,10.000000,357.0,324.0,33.0,0.09,0.00,0.91
3,6818277,silver,historic,1,1,5,36.000000,0.000000,0.000000,0.000000,360.0,351.0,9.0,0.00,0.21,0.79
4,8019088,gold,historic,2,1,51,36.000000,0.500006,2.500030,4.500055,2736.0,2523.0,213.0,0.06,0.04,0.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,1305122305,NaN,historic,7,2,17,29.142857,5.000000,5.000069,18.600000,74.0,72.0,2.0,0.00,0.00,1.00
282,1307920586,NaN,historic,1,1,5,12.000000,0.000000,0.000000,0.000000,95.0,60.0,35.0,1.00,0.00,0.00
283,1317128962,NaN,historic,1,1,5,12.000000,0.000000,0.000000,0.000000,167.0,135.0,32.0,1.00,0.00,0.00
284,1345286920,NaN,historic,1,1,25,36.000000,0.000000,0.000000,0.000000,256.0,235.0,21.0,0.33,0.00,0.67


In [ ]:
sellers_cat=np.unique(df_items.seller_id)
df_reputation=pd.DataFrame()
for i in sellers_cat:
    url = f'https://api.mercadolibre.com/users/{i}'
    rep = requests.get(url)
    reputation_tmp = rep.json()
    if isinstance(reputation_tmp, dict):
        df_reputation = pd.concat([df_reputation, pd.DataFrame([reputation_tmp])], axis=0)
    elif isinstance(reputation_tmp, list):
        df_reputation = pd.concat([df_reputation, pd.DataFrame(reputation_tmp)], axis=0)


In [ ]:
df_reputation=(
    df_reputation
    .reset_index(drop=True)
    .reindex(columns=['id','registration_date'])
    .assign(
        registration_date=lambda df: pd.to_datetime(df['registration_date']).dt.strftime('%Y-%m-%d'),
        years_antiquity=lambda df: (dt.datetime.today() - pd.to_datetime(df['registration_date'])).dt.days/ 365.25
        )
)      